# Importing Libraries

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.models import LdaModel
from gensim import models, corpora, similarities
import re
import time
from nltk import FreqDist
from scipy.stats import entropy
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
from pymongo import MongoClient
from nltk.tokenize import word_tokenize
import pickle
import string
from string import punctuation

In [2]:
#Details for getting data from projectfinder
db_loc = {
    'ip' :'10.10.250.0',
    'port' : 27017,
    'database' : 'projectfinder',
    'collection' : 'itproject_clean'
}

In [3]:
#Details for storing data related to projectfinder
db_data = {
    'ip' :'10.10.250.0',
    'port' : 27017,
    'database' : 'projectfinder',
    'collection' : 'mldata1'
}

In [4]:
def load_dataset_from_momgodb(db_obj):
    
    """
    This method loads a dataset as a pandas dataframe from MongoDB 
    
    Parameters:
    @db_obj (dict): Storing the ip address, port number, database name and collection name for dataset to be loaded
    
    Returns:
    panadas dataframe: Containing the loaded dataset
    """
    
    #Extracting the items from the inputted dictionary
    dbname = db_obj['database']
    ip = db_obj['ip']
    port = db_obj['port']
    collection = db_obj['collection']
    
    #Creating a connection to the database using MongoClient
    connection = MongoClient(ip, port)
    db = connection[dbname]
    
    #Excluding the fileds which are not needed in the dataframe 
    #Currenlty excluding the id associated with each document of the collection
    
    exclude_field = {'_id': False}
    raw_dataset = list(db[collection].find({}, projection=exclude_field))
    
    dataset = pd.DataFrame(raw_dataset)
    print(f'Data loaded from mongodb {collection} collection succesfully')
    return dataset

In [5]:
def save_to_momgodb(df,db_):
    
    """
    This method saves a dataframe as a collection into a specified MongoDB database.
    
    Parameters:
    @df (pandas dataframe): Storing the dataset to be saved
    @db_ (dict): Details for the database where the given dataset is to be saved
    
    """
    
    #Convert data prsent in the dataframe to JSON format
    data = df.to_dict(orient='records')
    
     #Extracting the items from the inputted dictionary of database details
    dbname = db_['database']
    ip = db_['ip']
    port = db_['port']
    coll = db_['collection']
    
    #Creating a connection to the database using MongoClient
    connection = MongoClient(ip,port)
    db = connection[dbname]
    col = db[collection].insert_many(data)
    
    print(f'data saved as {coll}')

In [6]:
def load_dataset_from_json(data):
    with open(data) as f:
            d = json.load(f)
        #normalize json
    dataset= json_normalize(d)
    return dataset

In [7]:
df = load_dataset_from_momgodb(db_loc)
df.shape

Data loaded from mongodb itproject_clean collection succesfully


(14059, 25)

In [8]:
def get_required_dataset(original_dataset):
    
    #Select required colunms
    df = original_dataset[['description', 'bereich']]
    df = df[df['description'] != '']
    #df.rename(columns = {'description' : 'project', 'bereich' : 'class'})
    df['project'] = df['description']
    df['label'] = df['bereich']
    df.drop(['description', 'bereich'], axis=1, inplace=True)
    df = df[df['label'] != 'IT/Bauingenieur']
    df = df.drop_duplicates()
    return df

In [9]:
df = get_required_dataset(df)
df.shape
df.head()

,project,label
0,Für einen unserer Kunden aus dem Finanzdienstl...,Infr-Admin-Microsoft
1,Kann Profil leider nicht löschen.,IT/Consulting
2,Business Intelligence Analyst (m/w) - Tableau ...,Data-Sci-BI
3,"Konzeption, Customizing sowie Softwareanpassun...",Dev-Web-Backend
4,Es sollen mehrere Automatisierungen mit ubot S...,IT/IT


In [10]:
# shuffle the data
df = df.sample(frac=1.0)
df.reset_index(drop=True,inplace=True)
df.head()

,project,label
0,Projektbeschreibung \n\n ...,ERP-SAP
1,Projektbeschreibung \n\n ...,Infr-Database-Admin
2,Aufgabe: \n* Erbringung von planerischen Leist...,IT/Weitere tech
3,Aufgabe: \n* Modifikationskonstruktionen auf B...,IT/Konstruktion
4,Projektbeschreibung \n\n ...,Dev-Web-Fullstack


In [11]:
df.iloc[0,0]

"Projektbeschreibung \n\n                             \n              Melden Sie sich jetzt an und bewerben Sie sich direkt! \n              Jetzt bewerben \n         \n    \n\n                                 SAP S2P (Ariba) Team Lead - Europe (Germany with some Remote working) - 12 months - c.€750/day   This is an opportunity starting at the beginning of February that will run for a full 12 months (with likely renewal).    This is on a long term English speaking global SAP deployment programme.   The working pattern is 4 days onsite and Friday remote. All expenses are paid for travel, flights and accommodation.   The role is for a Team Lead across S2P (SAP Ariba) - requiring both technical and functional expertise for Source-to-Pay (both upstream and downstream appreciation). You'll need experience across S2P processes, organisations and systems gained in procurement consulting roles.   KEY EXPERIENCE REQUIRED:     Strong experience across SAP S2P roll outs   SAP Ariba experience (up

In [12]:
def tokenization_data(text):
    text = re.sub(r'(\d)',' ',text.lower())
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    tokens = word_tokenize(text)
    return tokens

In [13]:
# Clean text and title and create new column "tokenized"
t1 = time.time()
df['tokenized'] = df['project'].apply(tokenization_data)
t2 = time.time()
print("Time taken to prepare", len(df), "projects documents:", (t2-t1)/60, "min")

Time taken to prepare 12130 projects documents: 0.3593214233716329 min


In [14]:
df.head()

,project,label,tokenized
0,Projektbeschreibung \n\n ...,ERP-SAP,"[projektbeschreibung, melden, sie, sich, jetzt..."
1,Projektbeschreibung \n\n ...,Infr-Database-Admin,"[projektbeschreibung, melden, sie, sich, jetzt..."
2,Aufgabe: \n* Erbringung von planerischen Leist...,IT/Weitere tech,"[aufgabe, erbringung, von, planerischen, leist..."
3,Aufgabe: \n* Modifikationskonstruktionen auf B...,IT/Konstruktion,"[aufgabe, modifikationskonstruktionen, auf, ba..."
4,Projektbeschreibung \n\n ...,Dev-Web-Fullstack,"[projektbeschreibung, melden, sie, sich, jetzt..."


In [15]:
stemmer_own = {
    
    'admin': 'administration',  
    'verwaltung': 'administration',
    'architektur' : 'architekture',
    'agil' : 'agile',
    'analys': 'analyst',
    'app': 'application',
    'anwend' : 'application',
    'automat': 'automate',
   
    
    'consultant' : 'berater',
    'berat': 'berater',
    'bereich' : 'bereich',
    'cisco': 'cisco',
    'konzept' : 'concept',
    'container': 'containerization',
    'zertifi' : 'certificate',
    'certifi' : 'certificate',
    'design' : 'design',
    'engineer' : 'engineer',
    'ingenieur'  : 'engineer',
    'entwick': 'entwicklung',
    'develop': 'entwicklung',
    'program': 'entwicklung',
    'entwickler' : 'entwicklung',
    
    'extern': 'external',
    'framework': 'framework',
    'globalen': 'global',
    'schnittstell': 'interface',
    'implement' : 'implementation', 
    'infrastr' : 'infrastructure',
    'informati' : 'informatik',
    'intern': 'internal',
    'manage' : 'management',
    'method' : 'method',
    'überwach' : 'monitoring',
    'mobil': 'mobil',
    'betrieb' : 'operation',
    
    'operat' : 'operation',
    'operie' : 'operation',
    'plattform' : 'platform',
    'projec' : 'project',
    'prozess' : 'process',
    'process' : 'process',
    'bearbeitung' : 'process',
    'scrum': 'scrum',
    'softwar': 'software',
    'spezifi' :'specification',
    'specifi' :'specification',
    'unterstützt' : 'support',
    'support' : 'support',
    'system': 'system',
    'anfoder': 'requirement',
    'tech' : 'tech',
    
}

In [16]:
stemmer_keys = list(stemmer_own.keys())
def stemming(words):
    stemmed = []
    for word in words:
        for stemmer_key in stemmer_keys:
            if stemmer_key in word:
                stemmed_word = stemmer_own[stemmer_key]
                stemmed.append(stemmed_word)
                break
        else:
            stemmed.append(word)
    return stemmed

In [18]:
df['tokenized_stemmed'] = df['tokenized'].apply(stemming)

In [19]:
df.head()

,project,label,tokenized,tokenized_stemmed
0,Projektbeschreibung \n\n ...,ERP-SAP,"[projektbeschreibung, melden, sie, sich, jetzt...","[projektbeschreibung, melden, sie, sich, jetzt..."
1,Projektbeschreibung \n\n ...,Infr-Database-Admin,"[projektbeschreibung, melden, sie, sich, jetzt...","[projektbeschreibung, melden, sie, sich, jetzt..."
2,Aufgabe: \n* Erbringung von planerischen Leist...,IT/Weitere tech,"[aufgabe, erbringung, von, planerischen, leist...","[aufgabe, erbringung, von, planerischen, leist..."
3,Aufgabe: \n* Modifikationskonstruktionen auf B...,IT/Konstruktion,"[aufgabe, modifikationskonstruktionen, auf, ba...","[aufgabe, modifikationskonstruktionen, auf, ba..."
4,Projektbeschreibung \n\n ...,Dev-Web-Fullstack,"[projektbeschreibung, melden, sie, sich, jetzt...","[projektbeschreibung, melden, sie, sich, jetzt..."


In [20]:
# load nltk's German and english stopwords'
import nltk
with open('../german_stopwords_full.txt', 'r') as f:
    stopwords_germ = f.read().splitlines()
stopwords_eng = nltk.corpus.stopwords.words('english')

#german cities
from bundeslander import Baden_Württemberg, Bayern, Berlin, Brandenburg, Bremen, Hamburg, Hessen, Mecklenburg_Vorpommern, Niedersachsen, Nordrhein_Westfalen, Rheinland_Pfalz, Saarland, Sachsen, Sachsen_Anhalt, Schleswig_Holstein, Thüringen, Ausland
All = Baden_Württemberg + Bayern + Berlin + Brandenburg + Bremen +Hamburg + Hessen + Mecklenburg_Vorpommern + Niedersachsen + Nordrhein_Westfalen + Rheinland_Pfalz + Saarland + Sachsen + Sachsen_Anhalt + Schleswig_Holstein + Thüringen + Ausland
cities = list(set([city.lower() for city in All]))

months = ['Januar', 'January','Februar', 'February', 'März', 'March', 'April', 'Mai', 'May', 'Juni', 'June', 'Juli', 
          'July', 'August', 'September', 'Oktober', 'October', 'November', 'Dezember', 'December']
months = [month.lower() for month in months]
print(months)

stopwords_manual = [line.rstrip('\n') for line in open('stopwords_manual.txt')]
print(len(stopwords_manual))

['januar', 'january', 'februar', 'february', 'märz', 'march', 'april', 'mai', 'may', 'juni', 'june', 'juli', 'july', 'august', 'september', 'oktober', 'october', 'november', 'dezember', 'december']
844


In [21]:
stopwords_all = list(set(stopwords_germ + stopwords_eng + stopwords_manual + cities + months))
len(stopwords_all)

13240

In [22]:
stopwords_add = []
stopwords_add = list(set(stopwords_add + stopwords_manual))
checker = list(set(stopwords_germ + stopwords_eng + cities + months))
stopwords_add.sort()
with open('stopwords_manual.txt', 'w') as f:
    for item in stopwords_add:
        if item not in checker:
            f.write("%s\n" % item)
stopwords_manual = [line.rstrip('\n') for line in open('stopwords_manual.txt')]
print(len(stopwords_manual))

844


In [23]:
stopwords_all = list(set(stopwords_germ + stopwords_eng + stopwords_manual + cities + months))
len(stopwords_all)

13240

In [24]:
def remove_stopwords(words):
    return [word for word in words if word not in stopwords_all]

In [25]:
df['token_stem_stopwordRem'] = df['tokenized_stemmed'].apply(remove_stopwords)

In [26]:
df.head()

,project,label,tokenized,tokenized_stemmed,token_stem_stopwordRem
0,Projektbeschreibung \n\n ...,ERP-SAP,"[projektbeschreibung, melden, sie, sich, jetzt...","[projektbeschreibung, melden, sie, sich, jetzt...","[sap, p, lead, europe, c, €, day, opportunity,..."
1,Projektbeschreibung \n\n ...,Infr-Database-Admin,"[projektbeschreibung, melden, sie, sich, jetzt...","[projektbeschreibung, melden, sie, sich, jetzt...","[rolling, beginning, utilization, ·, least, sk..."
2,Aufgabe: \n* Erbringung von planerischen Leist...,IT/Weitere tech,"[aufgabe, erbringung, von, planerischen, leist...","[aufgabe, erbringung, von, planerischen, leist...","[erbringung, planerischen, leistungen, ausschr..."
3,Aufgabe: \n* Modifikationskonstruktionen auf B...,IT/Konstruktion,"[aufgabe, modifikationskonstruktionen, auf, ba...","[aufgabe, modifikationskonstruktionen, auf, ba...","[modifikationskonstruktionen, vorhandener, tec..."
4,Projektbeschreibung \n\n ...,Dev-Web-Fullstack,"[projektbeschreibung, melden, sie, sich, jetzt...","[projektbeschreibung, melden, sie, sich, jetzt...","[project, management, bss, oss, sap, brim, hp,..."


In [27]:
bigram = gensim.models.Phrases(df['tokenized_stemmed'].tolist(), min_count=5, threshold=50) # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)
def make_bigrams(text):
    return bigram_mod[text]

In [28]:
# Form Bigrams
df['token_stem_bigram'] = df['token_stem_stopwordRem'].apply(make_bigrams)

In [29]:
df.head()

,project,label,tokenized,tokenized_stemmed,token_stem_stopwordRem,token_stem_bigram
0,Projektbeschreibung \n\n ...,ERP-SAP,"[projektbeschreibung, melden, sie, sich, jetzt...","[projektbeschreibung, melden, sie, sich, jetzt...","[sap, p, lead, europe, c, €, day, opportunity,...","[sap, p, lead, europe, c, €, day, opportunity,..."
1,Projektbeschreibung \n\n ...,Infr-Database-Admin,"[projektbeschreibung, melden, sie, sich, jetzt...","[projektbeschreibung, melden, sie, sich, jetzt...","[rolling, beginning, utilization, ·, least, sk...","[rolling, beginning, utilization, ·, least, sk..."
2,Aufgabe: \n* Erbringung von planerischen Leist...,IT/Weitere tech,"[aufgabe, erbringung, von, planerischen, leist...","[aufgabe, erbringung, von, planerischen, leist...","[erbringung, planerischen, leistungen, ausschr...","[erbringung, planerischen, leistungen, ausschr..."
3,Aufgabe: \n* Modifikationskonstruktionen auf B...,IT/Konstruktion,"[aufgabe, modifikationskonstruktionen, auf, ba...","[aufgabe, modifikationskonstruktionen, auf, ba...","[modifikationskonstruktionen, vorhandener, tec...","[modifikationskonstruktionen, vorhandener, tec..."
4,Projektbeschreibung \n\n ...,Dev-Web-Fullstack,"[projektbeschreibung, melden, sie, sich, jetzt...","[projektbeschreibung, melden, sie, sich, jetzt...","[project, management, bss, oss, sap, brim, hp,...","[project, management, bss, oss, sap, brim, hp,..."


In [34]:
# Obtain all bigrams
all_lines = df['token_stem_bigram'].tolist()
all_bigrams = list(set([word for words in all_lines for word in words if '_' in word]))
all_bigrams.sort()
print(len(all_bigrams))

2576


In [35]:
with open('all_bigrams_Approach_3.txt', 'w') as f:
    for item in all_bigrams:
        f.write("%s\n" % item)